<a href="https://colab.research.google.com/github/smokett/Intro_To_ML_cw1/blob/master/IntroToMLCW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Step 1: Importing data

In [4]:
#  importing data from files

clean_data = np.loadtxt('wifi_db/clean_dataset.txt')
noisy_data = np.loadtxt('wifi_db/noisy_dataset.txt')
print(clean_data.shape)
print(clean_data)

(2000, 8)
[[-64. -56. -61. ... -82. -81.   1.]
 [-68. -57. -61. ... -85. -85.   1.]
 [-63. -60. -60. ... -85. -84.   1.]
 ...
 [-62. -59. -46. ... -87. -88.   4.]
 [-62. -58. -52. ... -90. -85.   4.]
 [-59. -50. -45. ... -88. -87.   4.]]


# Step 2: Creating Decision Trees

## 2.1 Defining the functions to calculate information Gain

Functions below will calculate the information gain after we split the dataset to `S_left`, `S_right`

In [5]:
# this part in the notebook calculates the information gain from entropy and reamainder after made the decision

def gain_function(s_all,s_left,s_right):
  '''
  This function calculates the information gain
  '''
  return entropy_function(s_all) - remainder_function(s_left,s_right) 
  

def entropy_function(dataset):
  '''
  This function calculates the entropy of the dataset
  '''
  if len(dataset) > 0:
    labels = dataset[:,-1]
    unique, count = np.unique(labels, return_counts=True)
    # print(unique,count)
    p = [number/len(labels) for number in count]
    # print(p)

    result = - np.dot(p,np.log2(p))
    return result


def remainder_function(s_left, s_right):
  '''
  This function calculate the remainder after made the decision
  '''

  result = entropy_function(s_left) * len(s_left)/(len(s_left)+len(s_right)) + entropy_function(s_right) * len(s_right)/(len(s_left)+len(s_right))
  return result

In [6]:
def decision_tree_learning(training_dataset, depth):
  
  pass

## 2.2 FIND_SPLIT function
This function will find the split that give us the most ***information gain(IG)***, IG is got from the defined function above. 

The general idea is that it will try to make a split in every **wifi** data (i.e.**wifi1**), using every value for each wifi as a threshold to split, and find the split that give us the most IG.

----------------------------------
In the function below, a `checked_value` list is implemented to let the program run faster. So basically There will be many duplicated values in each wifi signals. What we will do is to `sort` the data first to get the value in order, and to check if the same value has been examined. In decision tree, there will be no need for us to do the same split for the same threshold. By having checking if the value has already been considered as a threshold, we are able to `make the split and calculate the IG only when encounter another unique value in the set`. By doing this we shortened our calculation process dramatically.(1999 --> 54)

In [7]:
def find_split(dataset):
  '''
  This function finds the split that will generate the highest information gain

  Input: dataset: this will be the input dataset, could be subsets that in leaf nodes
          The dataset is in dimension(...,8), with the last row element the label
  
  Output: S_left, S_left: this seperates the dataset into two parts that will generate the highest information gain
  '''
  # each row in wifis will be a wifi emitter's attribute, [:,-1] takes away the label
 
  # set the initial values
  highest_gain = -1
  best_cut = None
  best_left = None
  best_right = None


  
  #for each column in the dataset
  for col_num in range(0,len(dataset[0])-1):
    # get the sorted dataset according to the corresponding col
    args = np.argsort(dataset[:,col_num])
    sorted_data = dataset[args]
    # initial checked_value
    checked_value = []
    
    # start from the second until the last, so that left_set and right_set will not be empty
    # the last split would be dataset[:-1],dataset[-1]
    for row_num in range(1,len(dataset)):
      value = sorted_data[row_num,col_num]
      if value not in checked_value:
        checked_value.append(value)
        left_set = sorted_data[:row_num]
        right_set = sorted_data[row_num:]
        if left_set is [] or right_set is []:
          print('There is an empty list!')
        IG = gain_function(dataset, left_set, right_set)
        
        if IG > highest_gain:
          best_split_number,best_split_col = sorted_data[row_num,col_num],col_num
          best_left = left_set
          best_right = right_set
          highest_gain = IG
    # This is for checking purpose
    # print(len(checked_value)) !!! remember we have excluted the first row, so the actual unique value in the list might be plus 1! 
  return best_left,best_right,best_split_number,best_split_col

  




In [10]:
s_left,s_right,cut,pos = find_split(clean_data)

In [11]:
################################## for testing purpose###########################
print(s_left.shape)
print(s_right.shape)
print(cut)
print('in column',pos)

(1012, 8)
(988, 8)
-54.0
in column 0


## 2.3 Node Class

This section we will create a tree class, which will take the s_left and s_right as its attrribtes, and also keep track of its depth.

In [8]:
class Node :
  '''
  This is the Node class, we will create a node everytime we make a split
  INPUT: 
          depth: the depth of the node
          threshold: the value, and its wifi number this node used to split the data
  '''
  def __init__(self,attribute,value,s_left,s_right,is_leaf):
    self.attribute = attribute
    self.value = value
    self.left = s_left
    self.right = s_right
    self.is_leaf = is_leaf

  def draw_node(self):
    pass


In [9]:
def decision_tree_learning_function(training_dataset,depth):
  labels = training_dataset[:,-1]
  uni_label,label_counts = np.unique(labels,return_counts=True)
  {}
  if len(uni_label) == 1:
    node = {'attr':None,
        'value':uni_label[0],
        'left':None,
        'right':None,
        'is_leaf':True}
    return node,depth
  else:
    s_l,s_r,threshold,col_pos = find_split(training_dataset)

    l_branch,l_depth = decision_tree_learning_function(s_l,depth+1)
    r_branch,r_depth = decision_tree_learning_function(s_r,depth+1)

    node = {'attr':col_pos,
        'value':threshold,
        'left':l_branch,
        'right':r_branch,
        'is_leaf':False}
    

    return node, max(l_depth,r_depth)


  
    
    


In [10]:
node_result,depth_result = decision_tree_learning_function(clean_data,0)

In [11]:
print(node_result)

{'attr': 0, 'value': -54.0, 'left': {'attr': 4, 'value': -59.0, 'left': {'attr': 3, 'value': -55.0, 'left': {'attr': 2, 'value': -55.0, 'left': {'attr': None, 'value': 1.0, 'left': None, 'right': None, 'is_leaf': True}, 'right': {'attr': 6, 'value': -85.0, 'left': {'attr': 4, 'value': -62.0, 'left': {'attr': 5, 'value': -85.0, 'left': {'attr': 0, 'value': -56.0, 'left': {'attr': None, 'value': 4.0, 'left': None, 'right': None, 'is_leaf': True}, 'right': {'attr': None, 'value': 3.0, 'left': None, 'right': None, 'is_leaf': True}, 'is_leaf': False}, 'right': {'attr': None, 'value': 1.0, 'left': None, 'right': None, 'is_leaf': True}, 'is_leaf': False}, 'right': {'attr': None, 'value': 4.0, 'left': None, 'right': None, 'is_leaf': True}, 'is_leaf': False}, 'right': {'attr': None, 'value': 1.0, 'left': None, 'right': None, 'is_leaf': True}, 'is_leaf': False}, 'is_leaf': False}, 'right': {'attr': 1, 'value': -50.0, 'left': {'attr': None, 'value': 3.0, 'left': None, 'right': None, 'is_leaf': Tr

In [12]:
node_result

{'attr': 0,
 'is_leaf': False,
 'left': {'attr': 4,
  'is_leaf': False,
  'left': {'attr': 3,
   'is_leaf': False,
   'left': {'attr': 2,
    'is_leaf': False,
    'left': {'attr': None,
     'is_leaf': True,
     'left': None,
     'right': None,
     'value': 1.0},
    'right': {'attr': 6,
     'is_leaf': False,
     'left': {'attr': 4,
      'is_leaf': False,
      'left': {'attr': 5,
       'is_leaf': False,
       'left': {'attr': 0,
        'is_leaf': False,
        'left': {'attr': None,
         'is_leaf': True,
         'left': None,
         'right': None,
         'value': 4.0},
        'right': {'attr': None,
         'is_leaf': True,
         'left': None,
         'right': None,
         'value': 3.0},
        'value': -56.0},
       'right': {'attr': None,
        'is_leaf': True,
        'left': None,
        'right': None,
        'value': 1.0},
       'value': -85.0},
      'right': {'attr': None,
       'is_leaf': True,
       'left': None,
       'right': None,
    